# Functions 

In [7]:
def sizeof_number(number):
    """
    format values per thousands : K-thousands, M-millions, B-billions. 
    
    parameters:
    -----------
    number is the number you want to format
    
    """
    if number != 0:
        for unit in ['','K','M', 'B']:
            if abs(number) < 1000:
                if float(number) == int(number):
                    return f"{int(number)}{unit}"
                else:
                    return f"{number}{unit}"
            number /= 1000
        return f"{number:6.2f}"
    else:
        return ''

# Plotting Gross Buy and Sell

In [90]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import FuncFormatter
# Load data and aggregate the volume for both sides
data = pd.read_csv(r'Z:\External Research\2023 Interns\BPIPE\res\3738 HK Trade Journal.csv')

data = data[(data['Action'] == 'Trade')] # Filter trade
group_broker = data.groupby(by=["Broker Name", 'Side'])['Size'].sum()
group_broker = group_broker.unstack(level=-1)
group_broker = group_broker.fillna(0).sort_values(by=group_broker.columns[-1], ascending=False)

In [ ]:
# Plotting
font_color = '#525252'
hfont = {'fontname':'Calibri'}
facecolor = 'white'
color_red = '#c14e4e'
color_blue = '#99C4E7'
index = group_broker.index
if group_broker.shape[1] == 2:
    column0 = group_broker['ASK']
    column1 = group_broker['BID']
    title0 = 'Ask \n Total Volumn: {:,}'.format(int(group_broker.sum()[0]))
    title1 = 'Bid \n Total Volumn: {:,}'.format(int(group_broker.sum()[1]))
elif group_broker.shape[1] == 1 and group_broker.columns[0] == "ASK":
    column0 = group_broker['ASK']
    title0 = 'Ask \n Total Volumn: {:,}'.format(int(group_broker.sum()[0]))
elif group_broker.shape[1] == 1 and group_broker.columns[0] == "BID":
    column0 = group_broker['BID']
    title0 = 'Bid \n Total Volumn: {:,}'.format(int(group_broker.sum()[0]))

if group_broker.shape[1] == 2:
    fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=2, sharey=True)
    fig.tight_layout() 
    bar0 = axes[0].barh(index, column1, align='center', color=color_blue, alpha=0.8, zorder=10) #TODO: alpha=0.8
    axes[0].set_title(title1, fontsize=18, pad=15, color=color_blue, **hfont)
    bar1 = axes[1].barh(index, column0, align='center', color=color_red, zorder=10)
    axes[1].set_title(title0, fontsize=18, pad=15, color=color_red, **hfont)

    current_value_1 = axes[0].get_xticks()
    current_value_2 = axes[1].get_xticks()
    axes[0].set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value_1])
    axes[1].set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value_2])
    
    for i, v in enumerate(column1):
        if v < current_value_1[0]:
            axes[0].text(current_value_1[0], i, str(sizeof_number(v)), color='black', ha='right', va='center', zorder=11)
        else:
            axes[0].text(v/2, i, str(sizeof_number(v)), color='black', ha='right', va='center', zorder=11)
    for i, v in enumerate(column0):
        axes[1].text(v/2, i, str(sizeof_number(v)), color='black', ha='left', va='center', zorder=11)

    axes[0].invert_xaxis() 
    plt.gca().invert_yaxis()
    axes[0].set(yticks=group_broker.index, yticklabels=group_broker.index)
    axes[0].yaxis.tick_left()
    axes[0].tick_params(axis='y', colors='black') # tick color

if group_broker.shape[1] == 1:
    fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=1)
    fig.tight_layout() 
    if group_broker.columns[0] == "ASK":
        bars = axes.barh(index, column0, align='center', color=color_blue, zorder=10)
        axes.set_title(title0, fontsize=18, pad=15, color=color_blue, **hfont)
    if group_broker.columns[0] == "BID":
        bars = axes.barh(index, column0, align='center', color=color_red, zorder=10)
        axes.set_title(title0, fontsize=18, pad=15, color=color_red, **hfont)
    for i, v in enumerate(column0):
        axes.text(v/2, i, str(sizeof_number(v)), color='black', ha='left', va='center', zorder=11)

    plt.gca().invert_yaxis()
    axes.set(yticks=group_broker.index, yticklabels=group_broker.index)
    axes.yaxis.tick_left()

def gross_buy_sell(data: pd.DataFrame, trade: pd.DataFrame):
    data = data[data['Action'] == 'Trade'] # Filter trade
    group_broker = data.groupby(by=["Broker Name", 'Side'])['Size'].sum()
    group_broker = group_broker.unstack(level=-1)
    group_broker = group_broker.fillna(0).sort_values(by=group_broker.columns[0], ascending=False)

    total_vol = trade[(trade['conditionCodes'] != 'IE') & (trade['conditionCodes'] != 'U')]['size'].sum()

    # Plotting
    hfont = {'fontname':'Calibri'}
    facecolor = 'white'
    color_red = '#c14e4e'
    color_blue = '#99C4E7'

    if group_broker.shape[1] == 2:
        title0 = 'Ask \n Total Volume: {:,} \n Volume Mapped: {:,}'.format(int(total_vol), int(group_broker.sum()[0])) # TODO: add % of volume mapped # TODO: rewrite into f-string
        title1 = 'Bid \n Total Volume: {:,} \n Volume Mapped: {:,}'.format(int(total_vol), int(group_broker.sum()[1]))
        unmapped = pd.DataFrame({'ASK': total_vol  - group_broker.sum()[0], 'BID':  total_vol  - group_broker.sum()[1]}, index=['Unmapped Brokers'])
        group_broker = pd.concat([group_broker, unmapped])
        column0 = group_broker['ASK']
        column1 = group_broker['BID']
        index = group_broker.index
    elif group_broker.shape[1] == 1 and group_broker.columns[0] == "ASK":
        title0 = 'Ask \n Total Volume: {:,} \n Volume Mapped: {:,}'.format(total_vol, int(group_broker.sum()[0]))
        unmapped = pd.DataFrame({'ASK': total_vol  - group_broker.sum()[0]}, index=['Unmapped Brokers'])
        group_broker = pd.concat([group_broker, unmapped])
        column0 = group_broker['ASK']
        index = group_broker.index
    elif group_broker.shape[1] == 1 and group_broker.columns[0] == "BID":
        title0 = 'Bid \n Total Volume: {:,} \n Volume Mapped: {:,}'.format(total_vol, int(group_broker.sum()[0]))
        unmapped = pd.DataFrame({'BID':  total_vol  - group_broker.sum()[0]}, index=['Unmapped Brokers'])
        group_broker = pd.concat([group_broker, unmapped])
        column0 = group_broker['BID']
        index = group_broker.index

    if group_broker.shape[1] == 2:
        fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=2, sharey=True)
        fig.tight_layout()
        axes[0].barh(index, column1, align='center', color=color_blue, alpha=0.8, zorder=10)
        axes[0].set_title(title1, fontsize=13, pad=30, color=color_blue, alpha=0.8, **hfont)
        axes[1].barh(index, column0, align='center', color=color_red, alpha=0.8, zorder=10)
        axes[1].set_title(title0, fontsize=13, pad=30, color=color_red, alpha=0.8, **hfont)

        current_value_1 = axes[0].get_xticks()
        current_value_2 = axes[1].get_xticks()
        axes[0].set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value_1])
        axes[1].set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value_2])

        for i, v in enumerate(column0):
            if v < current_value_2[-1]*0.05:
                axes[1].text(current_value_2[-1]*0.05, i, str(sizeof_number(v)), color='black', ha='left', va='center', zorder=11)
            else:
                axes[1].text(v/2, i, str(sizeof_number(v)), color='black', ha='left', va='center', zorder=11)
        for i, v in enumerate(column1):
            if v < current_value_1[-1]*0.05:
                axes[0].text(current_value_1[-1]*0.05, i, str(sizeof_number(v)), color='black', ha='right', va='center', zorder=11)
            else:
                axes[0].text(v/2, i, str(sizeof_number(v)), color='black', ha='right', va='center', zorder=11)

        axes[0].invert_xaxis()
        plt.gca().invert_yaxis()
        axes[0].set(yticks=group_broker.index, yticklabels=group_broker.index)
        axes[0].yaxis.tick_left()
        axes[0].tick_params(axis='y', colors='black') # tick color

    elif group_broker.shape[1] == 1:
        fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=1)
        fig.tight_layout()
        if group_broker.columns[0] == "ASK":
            axes.barh(index, column0, align='center', color=color_blue, alpha=0.8, zorder=10)
            axes.set_title(title0, fontsize=18, pad=15, color=color_blue, alpha=0.8, **hfont)
        if group_broker.columns[0] == "BID":
            axes.barh(index, column0, align='center', color=color_red, alpha=0.8, zorder=10)
            axes.set_title(title0, fontsize=18, pad=15, color=color_red, alpha=0.8, **hfont)

        current_value = axes.get_xticks()
        axes.set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value])

        for i, v in enumerate(column0):
            if v < current_value[-1]*0.05:
                axes.text(current_value[-1]*0.05, i, str(sizeof_number(v)), color='black', ha='left', va='center', zorder=11)
            else:
                axes.text(v/2, i, str(sizeof_number(v)), color='black', ha='left', va='center', zorder=11)

        plt.gca().invert_yaxis()
        axes.set(yticks=group_broker.index, yticklabels=group_broker.index)
        axes.yaxis.tick_left()
        axes.tick_params(axis='y', colors='black') # tick color

    return fig



# Plotting with Active

In [85]:
import matplotlib.pyplot as plt
import pandas as pd

# Load data and aggregate the volume for both sides
data = pd.read_csv(r'Z:\External Research\2023 Interns\BPIPE\res\3738 HK Trade Journal with Active.csv')
data = data[(data['Action'] == 'Trade')].reset_index(drop=True) # Filter trade

group_active = data.groupby(by=['Broker Name', 'Side', 'Trade Type'])['Size'].sum()
group_active = group_active.unstack(level=-1)
group_active['Active'] = group_active.apply(lambda x: x['Automatch'] + x['Active'] if not pd.isna(x['Automatch']) else x['Active'], axis=1)
group_active['Passive'] = group_active.apply(lambda x: x['Automatch'] + x['Passive'] if not pd.isna(x['Automatch']) else x['Passive'], axis=1)
group_active = group_active.drop(columns=['Automatch'])
group_active = group_active.unstack(level=-1)
group_active = group_active.fillna(0).sort_values(by=group_active.columns[-1], ascending=False)

# group_active = group_active[[("Active", 'ASK'), ('Passive', 'ASK')]]

In [ ]:
# Plotting
hfont = {'fontname':'Calibri'}
facecolor = 'white'
color_red = '#CB7373'
color_blue = '#99C4E7'
color_blue_dark = '#529AD6'
color_red_dark = '#A23C3C'
index = group_active.index
width = 0.2
if group_active.shape[1] == 4:
    group_active['ASK'] = group_active.apply(lambda x: x[('Active', 'ASK')] +  x[('Passive', 'ASK')], axis=1)
    group_active['BID'] = group_active.apply(lambda x: x[('Active', 'BID')] +  x[('Passive', 'BID')], axis=1)  
    group_active = group_active.sort_values(by=group_active.columns[-1], ascending=False)
    column0 = group_active['ASK']
    column1 = group_active['BID']
    column01 = group_active[('Active', 'ASK')]
    column11 = group_active[('Active', 'BID')]
    title0 = 'Ask \n Total Volume: {:,}'.format(int(group_active['ASK'].sum()))
    title1 = 'Bid \n Total Volume: {:,}'.format(int(group_active['BID'].sum()))
elif group_active.shape[1] == 2 and group_active.columns[0] == ('Active', "ASK"):
    group_active['ASK'] = group_active.apply(lambda x: x[('Active', 'ASK')] +  x[('Passive', 'ASK')], axis=1)
    group_active = group_active.sort_values(by=group_active.columns[-1], ascending=False)
    column0 = group_active['ASK']
    column01 = group_active[('Active', 'ASK')]
    title0 = 'Ask \n Total Volume: {:,}'.format(int(group_active['ASK'].sum()))
elif group_active.shape[1] == 2 and group_active.columns[0] == ('Active', "BID"):
    group_active['BID'] = group_active.apply(lambda x: x[('Active', 'BID')] +  x[('Passive', 'BID')], axis=1)   
    group_active = group_active.sort_values(by=group_active.columns[-1], ascending=False)    
    column0 = group_active['BID']
    column01 = group_active[('Active', 'BID')]
    title0 = 'Bid \n Total Volume: {:,}'.format(int(group_active['BID'].sum()))

if group_active.shape[1] == 6:
    fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=2, sharey=True)
    fig.tight_layout() 
    axes[0].barh([x-width for x in range(group_active.shape[0])], column1, align='center', color=color_blue, alpha=0.8, zorder=10, height=0.4)
    axes[0].barh([x+width for x in range(group_active.shape[0])], column11, align='center', color=color_blue_dark, alpha=0.8, zorder=10, height=0.4)
    # axes[0].barh(index, column1, align='center', color=color_blue, alpha=0.8, zorder=10)
    # axes[0].barh(index, column11, align='center', color=color_blue_dark, alpha=0.8, zorder=10)
    axes[0].set_title(title1, fontsize=18, pad=15, color=color_blue, alpha=0.8, **hfont)
    axes[0].legend(['Total', 'Active'], loc='lower left')
    axes[1].barh([x-width for x in range(group_active.shape[0])], column0, align='center', color=color_red, alpha=0.8, zorder=10, height=0.4)
    axes[1].barh([x+width for x in range(group_active.shape[0])], column01, align='center', color=color_red_dark, alpha=0.8, zorder=10, height=0.4)
    # axes[1].barh(index, column0, align='center', color=color_red, alpha=0.8, zorder=10)
    # axes[1].barh(index, column01, align='center', color=color_red_dark, alpha=0.8, zorder=10)
    axes[1].set_title(title0, fontsize=18, pad=15, color=color_red, alpha=0.8, **hfont)
    axes[1].legend(['Total', 'Active'], loc='lower right')

    current_value_1 = axes[0].get_xticks()
    current_value_2 = axes[1].get_xticks()
    axes[0].set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value_1])
    axes[1].set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value_2])

    # for i, v in enumerate(column0):
    #     if v < current_value_2[-1]*0.05:
    #         axes[1].text(current_value_2[-1]*0.05, i, sizeof_number(v) + "\n" + sizeof_number(column01.iat[i]), color='black', ha='left', va='center', zorder=11)
    #     else:
    #         axes[1].text(v/2, i, sizeof_number(v) + '\n' + sizeof_number(column01.iat[i]), color='black', ha='left', va='center', zorder=11)

    # for i, v in enumerate(column1):
    #     if v < current_value_1[-1]*0.05:
    #         axes[0].text(current_value_1[-1]*0.05, i, sizeof_number(v) + "\n" + sizeof_number(column11.iat[i]), color='black', ha='right', va='center', zorder=11)
    #     else:
    #         axes[0].text(v/2, i, sizeof_number(v) + "\n" + sizeof_number(column11.iat[i]), color='black', ha='right', va='center', zorder=11)
    for i, v in enumerate(column0):
        if v < current_value_2[-1]*0.05:
            axes[1].text(current_value_2[-1]*0.05, i-width, sizeof_number(v), color='black', ha='left', va='center', zorder=11)
        else:
            axes[1].text(v/2, i-width, sizeof_number(v), color='black', ha='left', va='center', zorder=11)
    for i, v in enumerate(column01):
        if v < current_value_2[-1]*0.05:
            axes[1].text(current_value_2[-1]*0.05, i+width, sizeof_number(v), color='black', ha='left', va='center', zorder=11)
        else:
            axes[1].text(v/2, i+width, sizeof_number(v), color='black', ha='left', va='center', zorder=11)
    for i, v in enumerate(column1):
        if v < current_value_1[-1]*0.05:
            axes[0].text(current_value_1[-1]*0.05, i-width, sizeof_number(v), color='black', ha='right', va='center', zorder=11)
        else:
            axes[0].text(v/2, i-width, sizeof_number(v), color='black', ha='right', va='center', zorder=11)
    for i, v in enumerate(column11):
        if v < current_value_1[-1]*0.05:
            axes[0].text(current_value_1[-1]*0.05, i+width, sizeof_number(v), color='black', ha='right', va='center', zorder=11)
        else:
            axes[0].text(v/2, i+width, sizeof_number(v), color='black', ha='right', va='center', zorder=11)

    axes[0].invert_xaxis() 
    plt.gca().invert_yaxis()
    axes[0].set(yticks=[x for x in range(group_active.shape[0])], yticklabels=group_active.index)
    axes[0].yaxis.tick_left()
    axes[0].tick_params(axis='y', colors='black') # tick color

elif group_active.shape[1] == 3:
    fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=1)
    fig.tight_layout() 
    if group_active.columns[0] == ('Active', "ASK"):
        axes.barh(index, column0, align='center', color=color_blue, alpha=0.8, zorder=10)
        axes.barh(index, column01, align='center', color=color_blue_dark, alpha=0.8, zorder=10)
        axes.set_title(title0, fontsize=18, pad=15, color=color_blue, alpha=0.8, **hfont)
        axes.legend(['Total', 'Active'], loc='lower right')
    if  group_active.columns[0] == ('Active', "BID"):
        axes.barh(index, column0, align='center', color=color_red, alpha=0.8, zorder=10)
        axes.barh(index, column01, align='center', color=color_red_dark, alpha=0.8, zorder=10)
        axes.set_title(title0, fontsize=18, pad=15, color=color_red, alpha=0.8, **hfont)
        axes.legend(['Total', 'Active'], loc='lower right')

    current_value = axes.get_xticks()
    axes.set_xticklabels(['{}'.format(sizeof_number(x)) for x in current_value])

    for i, v in enumerate(column0):
        if v < current_value[-1]*0.05:
            axes.text(current_value[-1]*0.05, i, sizeof_number(v) + "\n" + sizeof_number(column01.iat[i]), color='black', ha='left', va='center', zorder=11)
        else:
            axes.text(v/2, i, sizeof_number(v) + '\n' + sizeof_number(column01.iat[i]), color='black', ha='left', va='center', zorder=11)
    

    plt.gca().invert_yaxis()
    axes.set(yticks=group_active.index, yticklabels=group_active.index)
    axes.yaxis.tick_left()
    axes.tick_params(axis='y', colors='black') # tick color

# Gross Buy and Sell with Unknowns

In [ ]:
def formatted_int(number):
    """
    format values per thousands : K-thousands, M-millions, B-billions.

    parameters:
    -----------
    number is the number you want to format

    """
    if number != 0:
        for unit in ['','K','M', 'B']:
            if abs(number) < 1000:
                if float(number) == int(number):
                    return f"{int(number)}{unit}"
                else:
                    return f"{number}{unit}"
            number /= 1000
        return f"{number:6.2f}"
    else:
        return ''
    

def gross_buy_sell(data: pd.DataFrame, trade: pd.DataFrame):

    data = data[data['Action'] == 'Trade'] # Filter trade
    group_broker = data.groupby(by=["Broker Name", 'Side'])['Size'].sum()
    group_broker = group_broker.unstack(level=-1)
    group_broker = group_broker.fillna(0).sort_values(by=group_broker.columns[0], ascending=False)

    total_vol = trade[(trade['conditionCodes'] != 'IE') & (trade['conditionCodes'] != 'U')]['size'].sum() # Calculate total volume from trades

    # Plotting
    hfont = {'fontname':'Calibri'}
    facecolor = 'white'
    color_dict = {'BID': '#99C4E7', 'ASK': '#c14e4e'}
    title_dict = {side: f'{side} \n Total Volume: {int(total_vol):,} \n Volume Mapped: {int(group_broker[side].sum()):,} \n Percentage Mapped: {group_broker[side].sum()/total_vol:.2%}' for side in group_broker.columns} # Set title
    # Add a new row of unmatched brokers
    unmapped = pd.DataFrame({side: total_vol  - group_broker[side].sum() for side in group_broker.columns}, index=['Unmapped Brokers'])
    group_broker = pd.concat([group_broker, unmapped]) 

    both_ask_and_bid = len(group_broker.columns) == 2 # If data contains both bid and ask side
    only_ask = (len(group_broker.columns) == 1) & ('ASK' in group_broker.columns) # If data contains only ask side
    
    if both_ask_and_bid:
        fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=2, sharey=True) # Create two subplots
        fig.tight_layout()
        
        for i, side in enumerate(group_broker.columns): # For both side
            # Set bars, title and axis labels 
            axes[i].barh(group_broker.index, group_broker[side], align='center', color=color_dict[side], zorder=10) 
            axes[i].set_title(title_dict[side], fontsize=18, pad=15, color=color_dict[side], **hfont)
            axes[i].set_xticklabels([f'{sizeof_number(x)}' for x in axes[i].get_xticks()]) 
            # Add bar labels
            largest_label = axes[i].get_xticks()[-1]
            for ix, vol in enumerate(group_broker[side]):
                if vol < largest_label * 0.05: # If the volume is lower than 5% of the largest axis-label, fix the label position; otherwise, center the label position
                    axes[i].text(largest_label * 0.05, ix, formatted_int(vol), color='black', ha='right', va='center', zorder=11)
                else:
                    axes[i].text(vol/2, ix, formatted_int(vol), color='black', ha='right', va='center', zorder=11)

        axes[0].invert_xaxis() # Put the bid to the left and invert the axis
        plt.gca().invert_yaxis() # Large volume at the top 
        axes[0].set(yticks=group_broker.index, yticklabels=group_broker.index) # Set broker's name in the left 
        axes[0].yaxis.tick_left()
        axes[0].tick_params(axis='y', colors='black') # tick color
        
    else: # Only ask or only bid
        fig, axes = plt.subplots(figsize=(10,15), facecolor=facecolor, ncols=1) # Create one plot
        fig.tight_layout()
        # Set bars, title and axis labels 
        axes.barh(group_broker.index, group_broker['ASK' if only_ask else 'BID'], align='center', color=color_dict['ASK' if only_ask else 'BID'], zorder=10)
        axes.set_title(title_dict['ASK' if only_ask else 'BID'], fontsize=18, pad=15, color=color_dict['ASK' if only_ask else 'BID'], **hfont)
        axes.set_xticklabels([f'{formatted_int(x)}' for x in axes.get_xticks()]) # Set axis label
        largest_label = axes.get_xticks()[-1]
        # Add bar labels
        for ix, vol in enumerate(group_broker['ASK' if only_ask else 'BID']):
            if vol < largest_label * 0.05:
                axes.text(largest_label * 0.05, ix, formatted_int(vol), color='black', ha='left', va='center', zorder=11)
            else:
                axes.text(vol/2, ix, formatted_int(vol), color='black', ha='left', va='center', zorder=11)

        plt.gca().invert_yaxis() # Large volume at the top 
        axes.set(yticks=group_broker.index, yticklabels=group_broker.index) # Set broker's name in the left 
        axes.yaxis.tick_left()
        axes.tick_params(axis='y', colors='black') # tick color

    return fig


# TOP after auction

In [ ]:
@staticmethod
def update_top(event, top_bid, top_ask):
    """
    Update the top broker book in the auction session; used in apply()

    event: Each row in top broker book in auction session
    """
    if event['MKTDEPTH_EVENT_SUBTYPE']=='BID':
        if event['MD_TABLE_CMD_RT'] == 'REPLACE':
            top_bid.loc[event['MBO_BID_POSITION_RT']] = [event['MBO_TIME_RT'], event['MBO_BID_POSITION_RT'], event['MBO_BID_RT'], event['MBO_BID_BROKER_RT']] # Replace the position in current top orderbook
        elif event['MD_TABLE_CMD_RT'] == 'REPLACE_CLEAR':
            top_bid.drop(event['MBO_BID_POSITION_RT']) # Clear the position in the top order book 
    else: # ASK
        if event['MD_TABLE_CMD_RT'] == 'REPLACE':
            top_ask.loc[event['MBO_ASK_POSITION_RT']] = [event['MBO_TIME_RT'], event['MBO_ASK_POSITION_RT'], event['MBO_ASK_RT'], event['MBO_ASK_BROKER_RT']] # Replace the position in current top orderbook
        elif event['MD_TABLE_CMD_RT'] == 'REPLACE_CLEAR':
            top_ask.drop(event['MBO_ASK_POSITION_RT']) # Clear the position in the top order book 

def concat_top_auction(self):
    """Concat top broker book after auction into top_update"""
    auc_time = self.mbo_update['MBO_TIME_RT'].iloc[0] # Time when auction session ends and MBO starts to update
    at_auction = self.top_update[.top_update.MBO_TIME_RT < auc_time] # TOP in the auction session
    top_bid = pd.DataFrame(columns=['MBO_TIME_RT', 'MBO_BID_POSITION_RT', 'MBO_BID_RT', 'MBO_BID_BROKER_RT'])
    top_ask = pd.DataFrame(columns=['MBO_TIME_RT', 'MBO_ASK_POSITION_RT', 'MBO_ASK_RT', 'MBO_ASK_BROKER_RT'])
    _ = at_auction.apply(self.update_top, axis=1, top_bid=top_bid, top_ask=top_ask) # Get top_bid and top_ask at the end of the auction session
    top_bid['MBO_TIME_RT'], top_ask['MBO_TIME_RT'] = self.get_time_to_second(auc_time), self.get_time_to_second(auc_time) 
    self.top_update = pd.concat([self.top_update, top_bid, top_ask])


# msg_parsing

In [ ]:
from typing import Literal
def add_del_mod(self, mbo:pd.Series, cmd: Literal["ADD", "DEL", "MOD"], side: Literal["BID", "ASK"]):
        """
        This function perform the command of ADD, DEL, or MOD to the current order book.
        ------
        mbo: Series, the change of orderbook
        cmd: str, one of "ADD", "DEL" and "MOD"
        side: str, one of "BID", "ASK"
        """

        df = self.df_bid.copy() if side == 'BID' else self.df_ask.copy()
        if cmd == 'DEL':
            df.drop(index=df[df[f'MBO_{side}_POSITION_RT'] == mbo[f'MBO_{side}_POSITION_RT']].index, inplace=True)
        elif cmd == 'ADD':
            if mbo[f'MBO_{side}_POSITION_RT'] not in df[f"MBO_{side}_POSITION_RT"]: # order book does not cover up to the broker position #FIXME: delete
                df.loc[len(df)] = [mbo[f'MBO_{side}_POSITION_RT'], mbo[f'MBO_{side}_RT'], "", "", mbo[f'MBO_{side}_SIZE_RT'], mbo['MBO_TIME_RT'], mbo['MD_TABLE_CMD_RT'], "TODO"]  # type: ignore
            else:
                df[f"MBO_{side}_POSITION_RT"] = df[f"MBO_{side}_POSITION_RT"].apply(lambda x: x + 1 if x >= mbo[f'MBO_{side}_POSITION_RT'] else x) # shift all positions by 1
                df.loc[len(df)] = [mbo[f'MBO_{side}_POSITION_RT'], mbo[f'MBO_{side}_RT'], "", "", mbo[f'MBO_{side}_SIZE_RT'], mbo['MBO_TIME_RT'], mbo['MD_TABLE_CMD_RT'], "TODO"]  # type: ignore
                df.sort_values(by=[f"MBO_{side}_POSITION_RT"], inplace=True)
        elif cmd == 'MOD':
            df.loc[df[df[f'MBO_{side}_POSITION_RT'] == mbo[f'MBO_{side}_POSITION_RT']].index, f"MBO_{side}_RT"] = mbo[f'MBO_{side}_RT']
            df.loc[df[df[f'MBO_{side}_POSITION_RT'] == mbo[f'MBO_{side}_POSITION_RT']].index, f"MBO_{side}_SIZE_RT"] = mbo[f'MBO_{side}_SIZE_RT']
        df.reset_index(drop=True)
        if side == 'BID':
             self.df_bid = df.reset_index(drop=True)
        elif side == 'ASK':
             self.df_ask = df.reset_index(drop=True)


In [16]:
def update_journal(self, side: Literal["BID", "ASK"], cmd: Literal["ADD", "DEL", "MOD"], mbo, top):
        """
        ...
        ------
        side: str, one of "BID", "ASK"
        cmd: str, one of "ADD", "DEL" and "MOD"
        mbo: Series, the change of orderbook
        top: DataFrame, the source of broker ID
        msg: list, stack all the results
        """

        if cmd == "DEL":
            df = self.df_bid.copy() if side == "BID" else self.df_ask.copy()
            self.journal.append(
                [
                    mbo["MBO_TIME_RT"],
                    df[df[f"MBO_{side}_POSITION_RT"] == mbo[f"MBO_{side}_POSITION_RT"]][f"MBO_{side}_BROKER_RT"].values[0],
                    mbo[f"MBO_{side}_RT"],
                    mbo["MKTDEPTH_EVENT_SUBTYPE"],
                    mbo[f"MBO_{side}_SIZE_RT"],  # qty = size of the whole order
                    mbo["MD_TABLE_CMD_RT"],
                    mbo[f"MBO_{side}_POSITION_RT"],
                ]
            )
            self.add_del_mod(mbo, cmd, side)
        
        elif cmd == "MOD":
            df = self.df_bid.copy() if side == "BID" else self.df_ask.copy()
            self.journal.append(
                [
                    mbo["MBO_TIME_RT"],
                    df[df[f"MBO_{side}_POSITION_RT"] == mbo[f"MBO_{side}_POSITION_RT"]][f"MBO_{side}_BROKER_RT"].values[0],
                    mbo[f"MBO_{side}_RT"],
                    mbo["MKTDEPTH_EVENT_SUBTYPE"],
                    df[df[f"MBO_{side}_POSITION_RT"] == mbo[f"MBO_{side}_POSITION_RT"]][f"MBO_{side}_SIZE_RT"].values[0] - mbo[f"MBO_{side}_SIZE_RT"],  # qty = change in size of the order
                    mbo["MD_TABLE_CMD_RT"],
                    mbo[f"MBO_{side}_POSITION_RT"],
                ]
            )
            self.add_del_mod(mbo, cmd, side)
        
        elif cmd == "ADD":
            self.add_del_mod(mbo, cmd, side)
            df = self.df_bid.copy() if side == "BID" else self.df_ask.copy()
            unmatched_orders = df[df[f"MBO_{side}_BROKER_RT"] == "0000"] # orders from umatched broker "0000"
            for ix, row in unmatched_orders.iterrows():
                query = f'(MBO_{side}_POSITION_RT == @row["MBO_{side}_POSITION_RT"]) & (MBO_{side}_RT == @row["MBO_{side}_RT"])' # Match position number and price
                broker_list = top.query(query, engine='python')[f"MBO_{side}_BROKER_RT"].to_list() # Find broker ID 
                if len(broker_list) > 0: # if we can match the broker
                    df.loc[ix, f"MBO_{side}_BROKER_RT"] = broker_list[-1]
            self.journal.append(
                [
                    mbo["MBO_TIME_RT"],
                    df[df[f"MBO_{side}_POSITION_RT"] == mbo[f"MBO_{side}_POSITION_RT"]][f"MBO_{side}_BROKER_RT"].values[0],
                    mbo[f"MBO_{side}_RT"],
                    mbo["MKTDEPTH_EVENT_SUBTYPE"],
                    mbo[f"MBO_{side}_SIZE_RT"],
                    mbo["MD_TABLE_CMD_RT"],
                    mbo[f"MBO_{side}_POSITION_RT"],
                ]
            )
            if side == "BID":
                self.df_bid = df
            elif side == "ASK":
                self.df_ask = df


In [ ]:
def match_trades(self):
        """ """

        df = self.journal_df[self.journal_df['Timestamp'].isin(self.trade_prints['time'])] # Timestamp filter
        df = df[df['Action'].isin(['DEL', 'MOD'])] # Action filter
        
        for ix, row in df.iterrows():
            trade = self.trade_prints[self.trade_prints['time'] == row['Timestamp']]
            for _, entry in trade.iterrows():
                if entry['value'] == row['Price'] and entry['size'] == row['Size'] and row['Position'] == 1:
                    self.journal_df.loc[ix, 'Action'] = 'Executed'
                    self.journal_df.loc[ix, "Condition Codes"] = entry["conditionCodes"]


In [ ]:
def concat_top_auction(self):
    """Create an initial paint of TOP at 9:20 (right after the auction) and concat to the `self.all_TOP_df`"""
    auc_time = self.all_MBO_df["MBO_TIME_RT"].iloc[0]  # Time when auction session ends and MBO starts to update
    TOP_df_during_auction = self.all_TOP_df[self.all_TOP_df.MBO_TIME_RT < auc_time]  # TOP in the auction session
    top_bid = pd.DataFrame(columns=["MBO_TIME_RT", "MBO_BID_POSITION_RT", "MBO_BID_RT", "MBO_BID_BROKER_RT"])
    top_ask = pd.DataFrame(columns=["MBO_TIME_RT", "MBO_ASK_POSITION_RT", "MBO_ASK_RT", "MBO_ASK_BROKER_RT"])
    
    for ix, event in TOP_df_during_auction.iterrows():
        side = event["MKTDEPTH_EVENT_SUBTYPE"]
        df = top_bid if side == 'BID' else top_ask
        if event["MD_TABLE_CMD_RT"] == "REPLACE":
            df.loc[event[f"MBO_{side}_POSITION_RT"]] = [event["MBO_TIME_RT"], event[f"MBO_{side}_POSITION_RT"], event[f"MBO_{side}_RT"], event[f"MBO_{side}_BROKER_RT"]]
        elif event["MD_TABLE_CMD_RT"] == "REPLACE_CLEAR":
            df.drop(event[f"MBO_{side}_POSITION_RT"], inplace=True)
        if side == 'BID':
            top_bid = df
        elif side == "ASK":
            top_ask = df
    
    top_bid["MBO_TIME_RT"], top_ask["MBO_TIME_RT"] = self.get_time_to_second(auc_time), self.get_time_to_second(auc_time)
    self.all_TOP_df = pd.concat([self.all_TOP_df, top_bid, top_ask])